# Test Tools and Models

## Running this notebook

To run this in VS Code:
1. Open a terminal
2. Navigate to `./backend`
3. Configure poetry to set up a venv environment by running `poetry config virtualenvs.in-project true`
4. Remove any existing poetry venv by running `poetry remove --all`
5. Run `poetry install` to install the dependencies in the virtual environment
6. In Select Kernel dropdown in the top right corner of VS Code, select `Python Environments` and select the venv environment you created at `backend/.venv/bin/python`
7. Run the notebook by clicking on the play button in the top right corner of VS Code or running `jupyter notebook test_notebook.ipynb`

In [ ]:
import sys
import os

# Get the absolute path to the src directory
src_path = os.path.abspath('src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

from dotenv import load_dotenv
load_dotenv('src/.env')
os.environ['MILVUS_HOST'] = 'localhost'
os.environ['OLLAMA_HOST'] = 'localhost'
from src.models import get_model, bind_tools, handle_tool_call, handle_streaming_tool_call
from src.tools import retrieve_context, multiply
from src.langgraph_agent import create_agent_graph, run_agent_graph, run_agent_graph_streaming
from src.app import ChatRequest, prompt_templates

In [ ]:
# test_notebook.ipynb (Revised)

async def test_agent_graph_streaming(user_prompt, agent_graph):

    # Run the agent and collect chunks
    collected = []
    async for chunk in run_agent_graph_streaming(agent_graph, user_prompt):
        collected.append(chunk)
        print("Chunk:", chunk)  # Print each chunk as it arrives

    final_response = "".join(collected)
    return final_response

# Execute the test in Jupyter:
assistant_system_prompt = prompt_templates["assistant_system_prompt"]
tools = [multiply, retrieve_context]
agent_graph = create_agent_graph(tools, assistant_system_prompt, True) # Need to set Agent Graph to True to enable streaming
# Test the agent graph with a user prompt
user_prompt = "What is the phone number for DataNinja Support?"
await test_agent_graph_streaming(user_prompt, agent_graph)  

In [ ]:
graph = agent_graph.get_graph().draw_mermaid()
# Display the graph in Jupyter

import base64
from IPython.display import Image, display

def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    return Image(url="https://mermaid.ink/img/" + base64_string)

img = mm(graph)

display(img)

In [ ]:
agent_graph = create_agent_graph(tools, assistant_system_prompt, False)
# Test the agent graph with a user prompt
response = run_agent_graph(agent_graph, user_prompt)
print('\n\n\n')
print(response)